## Problem Set 3 (Part 2 of 2): Interacting with the Alpha Vantage API using Python libraries

In Part 2 of Problem Set 3 we will be using the data retrived from the Alpha Vantage API and creating an optimized portfolio of stocks based on Modern Portfolio Theory. The PyPortfolioOpt package will be used to optimize a portfolio of stocks based maximizing the Sharpe Ratio. This exercise is an example of how a robo-advisor would use data APIs to continuously monitor and adjusts stock portfolios. 

This part of the exercise demonstrates:

1. Setting up a programming environment to work with APIs
2. Data preprocessing to create compatible datasets
3. Using a quantititive popular portfolio optimization library to estimate risks and weights

Documentation links:
<br />Alpha Vantage: https://www.alphavantage.co/documentation/
<br />Alpha Vantage Python library: https://alpha-vantage.readthedocs.io/en/latest/#
<br />Pandas DataFrame: https://pandas.pydata.org/pandas-docs/stable/reference/frame.html
<br />PyPortfolioOpt: https://pyportfolioopt.readthedocs.io/en/latest/index.html

In [13]:
api_key = "1XE2UKG2DV1AFGG9"

In [14]:
import pandas as pd
import requests
from alpha_vantage.timeseries import TimeSeries
import matplotlib.pyplot as plt

# PyPortfolioOpt libraries
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

In [15]:
ts = TimeSeries(key=api_key,output_format='pandas')

In [16]:
fb_data, fb_meta_data = ts.get_daily(symbol='FB', outputsize='full')
msft_data, msft_meta_data = ts.get_daily(symbol='MSFT', outputsize='full')
aapl_data, aapl_meta_data = ts.get_daily(symbol='AAPL', outputsize='full')
amzn_data, amzn_meta_data = ts.get_daily(symbol='AMZN', outputsize='full')

Isolate "close" column of data for each security:

In [19]:
fb_clean = fb_data['4. close']
msft_clean = msft_data['4. close']
aapl_clean = aapl_data['4. close']
amzn_clean = amzn_data['4. close']

Create new dataframe with tickers as columns:

In [20]:
df = pd.DataFrame(columns=['fb', 'msft', 'aapl', 'amzn'])

In [26]:
df

,fb,msft,aapl,amzn
date,,,,
2012-05-18,38.232,38.232,38.232,38.232
2012-05-21,34.030,34.030,34.030,34.030
2012-05-22,31.000,31.000,31.000,31.000
2012-05-23,32.000,32.000,32.000,32.000
2012-05-24,33.030,33.030,33.030,33.030
2012-05-25,31.910,31.910,31.910,31.910
2012-05-29,28.840,28.840,28.840,28.840
2012-05-30,28.190,28.190,28.190,28.190
2012-05-31,29.600,29.600,29.600,29.600


Place data into new dataframe:

In [27]:
df['fb'] = fb_clean
df['msft'] = fb_clean
df['aapl'] = fb_clean
df['amzn'] = fb_clean
print(df)

                 fb     msft     aapl     amzn
date                                          
2012-05-18   38.232   38.232   38.232   38.232
2012-05-21   34.030   34.030   34.030   34.030
2012-05-22   31.000   31.000   31.000   31.000
2012-05-23   32.000   32.000   32.000   32.000
2012-05-24   33.030   33.030   33.030   33.030
2012-05-25   31.910   31.910   31.910   31.910
2012-05-29   28.840   28.840   28.840   28.840
2012-05-30   28.190   28.190   28.190   28.190
2012-05-31   29.600   29.600   29.600   29.600
2012-06-01   27.720   27.720   27.720   27.720
2012-06-04   26.900   26.900   26.900   26.900
2012-06-05   25.869   25.869   25.869   25.869
2012-06-06   26.810   26.810   26.810   26.810
2012-06-07   26.310   26.310   26.310   26.310
2012-06-08   27.100   27.100   27.100   27.100
2012-06-11   27.005   27.005   27.005   27.005
2012-06-12   27.400   27.400   27.400   27.400
2012-06-13   27.270   27.270   27.270   27.270
2012-06-14   28.290   28.290   28.290   28.290
2012-06-15   

Take only the earlist half of observations:

In [28]:
df = df[0:924]

In [29]:
print(df)

                 fb     msft     aapl     amzn
date                                          
2012-05-18   38.232   38.232   38.232   38.232
2012-05-21   34.030   34.030   34.030   34.030
2012-05-22   31.000   31.000   31.000   31.000
2012-05-23   32.000   32.000   32.000   32.000
2012-05-24   33.030   33.030   33.030   33.030
2012-05-25   31.910   31.910   31.910   31.910
2012-05-29   28.840   28.840   28.840   28.840
2012-05-30   28.190   28.190   28.190   28.190
2012-05-31   29.600   29.600   29.600   29.600
2012-06-01   27.720   27.720   27.720   27.720
2012-06-04   26.900   26.900   26.900   26.900
2012-06-05   25.869   25.869   25.869   25.869
2012-06-06   26.810   26.810   26.810   26.810
2012-06-07   26.310   26.310   26.310   26.310
2012-06-08   27.100   27.100   27.100   27.100
2012-06-11   27.005   27.005   27.005   27.005
2012-06-12   27.400   27.400   27.400   27.400
2012-06-13   27.270   27.270   27.270   27.270
2012-06-14   28.290   28.290   28.290   28.290
2012-06-15   

Calculate expected returns:

In [30]:
mu = expected_returns.mean_historical_return(df)

Calculate sample covariance matrix:

In [32]:
S = risk_models.sample_cov(df)

Optimize the weights to maximzie the Sharpe Ratio:

In [33]:
ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe()
ef.portfolio_performance(verbose=True)

Expected annual return: 33.5%
Annual volatility: 42.8%
Sharpe Ratio: 0.74


(0.3354098691329444, 0.42809269212937484, 0.7367793819699769)

In [34]:
print(weights)

{'fb': 0.25, 'msft': 0.25, 'aapl': 0.25, 'amzn': 0.25}


Using optimized weights, compute portfolio accolation:

In [35]:
latest_prices = get_latest_prices(df)
da = DiscreteAllocation(weights, latest_prices, total_portfolio_value=1000000)
allocation, leftover = da.greedy_portfolio()
print(allocation)

0 out of 4 tickers were removed
{'fb': 2655, 'msft': 2655, 'aapl': 2655, 'amzn': 2655}


To update portfolio, call the API again for latest data and run optimization model again:

In [39]:
fb_data, fb_meta_data = ts.get_daily(symbol='FB', outputsize='full')
msft_data, msft_meta_data = ts.get_daily(symbol='MSFT', outputsize='full')
aapl_data, aapl_meta_data = ts.get_daily(symbol='AAPL', outputsize='full')
amzn_data, amzn_meta_data = ts.get_daily(symbol='AMZN', outputsize='full')

In [40]:
df = pd.DataFrame(columns=['fb', 'msft', 'aapl', 'amzn'])

In [42]:
fb_clean = fb_data['4. close']
msft_clean = msft_data['4. close']
aapl_clean = aapl_data['4. close']
amzn_clean = amzn_data['4. close']

In [43]:
df['fb'] = fb_clean
df['msft'] = msft_clean
df['aapl'] = aapl_clean
df['amzn'] = amzn_clean
print(df)

                 fb     msft     aapl      amzn
date                                           
2012-05-18   38.232   29.270  530.380   213.850
2012-05-21   34.030   29.750  561.280   218.110
2012-05-22   31.000   29.760  556.970   215.330
2012-05-23   32.000   29.110  570.560   217.280
2012-05-24   33.030   29.070  565.320   215.240
2012-05-25   31.910   29.060  562.290   212.890
2012-05-29   28.840   29.560  572.270   214.750
2012-05-30   28.190   29.340  579.170   209.230
2012-05-31   29.600   29.190  577.730   212.910
2012-06-01   27.720   28.450  560.990   208.220
2012-06-04   26.900   28.550  564.290   214.570
2012-06-05   25.869   28.505  562.830   213.210
2012-06-06   26.810   29.350  571.460   217.640
2012-06-07   26.310   29.230  571.720   218.800
2012-06-08   27.100   29.650  580.320   218.480
2012-06-11   27.005   28.895  571.170   216.500
2012-06-12   27.400   29.290  576.160   216.420
2012-06-13   27.270   29.130  572.160   214.730
2012-06-14   28.290   29.340  571.530   

In [44]:
mu = expected_returns.mean_historical_return(df)
S = risk_models.sample_cov(df)
ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe()
ef.portfolio_performance(verbose=True)
print(weights)

Expected annual return: 28.5%
Annual volatility: 22.3%
Sharpe Ratio: 1.19
{'fb': 0.14110788084764556, 'msft': 0.42339738777063635, 'aapl': 0.0, 'amzn': 0.43549473138171824}


In [45]:
latest_prices = get_latest_prices(df)
da = DiscreteAllocation(weights, latest_prices, total_portfolio_value=1000000)
allocation, leftover = da.greedy_portfolio()
print(allocation)

1 out of 4 tickers were removed
{'amzn': 246, 'msft': 3038, 'fb': 772}
